In [2]:
import numpy as np
import pandas as pd
import os
import time
import copy

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data as data_utils
from torchvision import datasets, models, transforms

from sklearn.model_selection import train_test_split

# For image-keypoints data augmentation
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

In [7]:
path='./data/'
train_path=os.path.join(path+'train_df.csv')
model_name='resnet'
mode_ver='18'

num_classes=48 
batch_size=64
num_epochs=50
num_spilts=10
num_earlystop=10
input_w=150
input_h=150
learning_rate=1e-2
feature_extract=False


In [8]:
df=pd.read_csv(train_path)
df.head()

,image,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,...,right_palm_x,right_palm_y,spine2(back)_x,spine2(back)_y,spine1(waist)_x,spine1(waist)_y,left_instep_x,left_instep_y,right_instep_x,right_instep_y
0,001-1-1-01-Z17_A-0000001.jpg,1046.389631,344.757881,1041.655294,329.820225,1059.429507,334.484230,1020.117796,338.890539,1048.000000,...,1067.000000,335.000000,1019.484230,455.000000,1026.515770,514.054730,998.578836,826.718013,1063.204067,838.827465
1,001-1-1-01-Z17_A-0000003.jpg,1069.850679,340.711494,1058.608552,324.593690,1075.242111,325.593690,1041.422997,331.694815,1065.593682,...,1081.187380,323.000000,1046.953248,454.062706,1058.766231,508.797029,1002.265676,699.062706,1066.376234,841.499445
2,001-1-1-01-Z17_A-0000005.jpg,1084.475902,337.000008,1078.717997,323.757889,1095.648412,325.242119,1061.039884,329.351571,1086.461032,...,1101.000000,334.000000,1044.538960,442.054730,1052.844144,495.890539,989.437847,808.757889,1066.071417,841.749554
3,001-1-1-01-Z17_A-0000007.jpg,1042.320047,361.452689,1037.907194,344.117804,1050.328382,353.913729,1016.844144,340.913737,1042.164191,...,1057.406318,372.461040,982.937294,458.109462,990.375124,507.624866,1001.305177,829.233767,1159.516499,599.389997
4,001-1-1-01-Z17_A-0000009.jpg,1058.046395,343.164191,1046.717997,331.703163,1058.132650,331.781079,1031.258806,338.593690,1049.812620,...,1069.648429,334.109461,1024.843791,453.687572,1034.391088,510.843791,998.625231,805.218921,1059.625956,839.765102


In [15]:
imgs=df.iloc[:,0].to_numpy()
motions=df.iloc[:,1:]
columns=motions.columns.to_list()[::2]
class_labels=[label.replace('_x','').replace('_y','')for label in columns]

keypoints=[]
for motion in motions.to_numpy():
    a_keypoints=list()
    for i in range(0,motion.shape[0],2):
        a_keypoints.append((float(motion[i]),float(motion[i+1])))
    keypoints.append(a_keypoints)
keypoints=np.array(keypoints)
print(imgs)
print(motions)
print(columns)


['001-1-1-01-Z17_A-0000001.jpg' '001-1-1-01-Z17_A-0000003.jpg'
 '001-1-1-01-Z17_A-0000005.jpg' ... '642-2-4-31-Z148_E-0000027.jpg'
 '642-2-4-31-Z148_E-0000029.jpg' '642-2-4-31-Z148_E-0000031.jpg']
           nose_x      nose_y   left_eye_x  left_eye_y  right_eye_x  \
0     1046.389631  344.757881  1041.655294  329.820225  1059.429507   
1     1069.850679  340.711494  1058.608552  324.593690  1075.242111   
2     1084.475902  337.000008  1078.717997  323.757889  1095.648412   
3     1042.320047  361.452689  1037.907194  344.117804  1050.328382   
4     1058.046395  343.164191  1046.717997  331.703163  1058.132650   
...           ...         ...          ...         ...          ...   
4190   637.251052  781.342260  1117.135818  370.667301  1097.123228   
4191   637.251052  781.342260  1117.135818  370.667301  1097.123228   
4192   637.251052  781.342260  1117.135818  370.667301  1097.123228   
4193   637.251052  781.342260  1117.135818  370.667301  1097.123228   
4194   637.251052  781

In [17]:
print(class_labels)
keypoints.shape
device=torch.device('cuda'if torch.cuda.is_available() else 'cpu')
device

['nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle', 'right_ankle', 'neck', 'left_palm', 'right_palm', 'spine2(back)', 'spine1(waist)', 'left_instep', 'right_instep']


device(type='cuda')